In [1]:
import numpy as np
import pandas as pd

In [2]:
bill_summaries = pd.read_csv('data/bill_summaries.csv')
bio_info = pd.read_csv('data/bioinfo.csv')
fec_ids = pd.read_csv('data/fec_ids.csv')
ideology = pd.read_csv('data/ideology.csv')
sponsored_legislation = pd.read_csv('data/sponsored_legislation.csv')
terms = pd.read_csv('data/terms.csv')
vote_compare = pd.read_csv('data/vote_compare.csv')
contrib = pd.read_csv('data/contrib.csv')


1. Do each of the CSVs have a primary key? If so, what?

In [3]:
bill_summaries[['bill.type', 'bill.number', 'versionCode']].duplicated().value_counts()

False    2751
Name: count, dtype: int64

In [4]:
bill_summaries[['bill.url', 'versionCode']].duplicated().value_counts()

False    2751
Name: count, dtype: int64

In [5]:
bio_info[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [6]:
fec_ids[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [7]:
ideology[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [8]:
sponsored_legislation[['url']].duplicated().value_counts()

False    14379
Name: count, dtype: int64

In [9]:
terms[['bioguide_id', 'chamber', 'congress']].duplicated().value_counts()

False    3257
Name: count, dtype: int64

In [10]:
vote_compare[['bioname', 'comparison_member']].duplicated().value_counts()

False    206040
Name: count, dtype: int64

In [11]:
contrib=contrib.drop_duplicates()

2. Do any of the tables need to become multiple tables, or combined into one table?

    * Should be combined into one table (share the same primary key): bioinfo, fec_ids, ideology

In [12]:
members = pd.merge(bio_info, fec_ids, 
                   how='outer', on='bioguide_id', 
                   validate='one_to_one', indicator='matched')

In [13]:
members['matched'].value_counts()

matched
both          545
left_only       0
right_only      0
Name: count, dtype: int64

In [14]:
members = members.drop(['matched'], axis=1)

In [15]:
members = pd.merge(members, ideology,
                   how='outer', on='bioguide_id', 
                   validate='one_to_one', indicator='matched')

In [16]:
members.columns

Index(['bioguide_id', 'Full name', 'Chamber', 'State', 'Party', 'District',
       'birthYear', 'image', 'Office address', 'Phone', 'Website', 'fec_id',
       'bioname', 'chamber', 'left_right_ideology', 'state_abbrev',
       'district_code', 'icpsr', 'party', 'matched'],
      dtype='object')

In [17]:
members = members.drop(['Chamber', 'Party', 'District', 'State', 'matched'], axis=1)

In [18]:
members.columns = [c.lower().replace(' ', '_') for c in members.columns]

In [19]:
members 

,bioguide_id,full_name,birthyear,image,office_address,phone,website,fec_id,bioname,chamber,left_right_ideology,state_abbrev,district_code,icpsr,party
0,A000055,Robert B. Aderholt,1965.0,https://www.congress.gov/img/member/a000055_20...,"272 Cannon House Office Building, Washington, ...",(202) 225-4876,https://aderholt.house.gov/,H6AL04098,"ADERHOLT, Robert",House,0.405,AL,4,29701,Republican
1,A000148,Jake Auchincloss,1988.0,https://www.congress.gov/img/member/67817e391f...,"1524 Longworth House Office Building, Washingt...",(202) 225-5931,https://auchincloss.house.gov,H0MA04192,"AUCHINCLOSS, Jake",House,-0.288,MA,4,22100,Democrat
2,A000369,Mark E. Amodei,1958.0,https://www.congress.gov/img/member/a000369_20...,"104 Cannon House Office Building, Washington, ...",(202) 225-6155,https://amodei.house.gov,H2NV02395,"AMODEI, Mark E.",House,0.384,NV,2,21196,Republican
3,A000370,Alma S. Adams,1946.0,https://www.congress.gov/img/member/a000370_20...,"2436 Rayburn House Office Building, Washington...",(202) 225-1510,https://adams.house.gov,H4NC12100,"ADAMS, Alma",House,-0.462,NC,12,21545,Democrat
4,A000371,Pete Aguilar,1979.0,https://www.congress.gov/img/member/a000371_20...,"108 Cannon House Office Building, Washington, ...",(202) 225-3201,https://aguilar.house.gov/,H2CA31125,"AGUILAR, Peter Rey",House,-0.324,CA,33,21506,Democrat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,W000830,George Whitesides,1974.0,https://www.congress.gov/img/member/68dc43db19...,"1504 Longworth House Office Building, Washingt...",(202) 225-1956,https://whitesides.house.gov/,H4CA27111,"WHITESIDES, George",House,-0.189,CA,27,22559,Democrat
541,W000831,James R. Walkinshaw,1982.0,https://www.congress.gov/img/member/68c1bd4ca9...,"2265 Rayburn House Office Building, Washington...",(202) 225-1492,https://walkinshaw.house.gov/,H6VA11066,"WALKINSHAW, James R.",House,-0.503,VA,11,22564,Democrat
542,Y000064,Todd Young,1972.0,https://www.congress.gov/img/member/y000064_20...,185 Dirksen Senate Office Building Washington...,(202) 224-5623,https://www.young.senate.gov,S0IN00194,"YOUNG, Todd",Senate,0.438,IN,0,21133,Republican
543,Y000067,Rudy Yakym III,1984.0,https://www.congress.gov/img/member/y000067_20...,"349 Cannon House Office Building, Washington, ...",(202) 225-3915,https://yakym.house.gov,H2IN02295,"YAKYM, Rudy, III",House,0.513,IN,2,22171,Republican


In [20]:
members.to_csv('data/thirdNF/members.csv', index=False)

In [21]:
for c in bill_summaries.columns: 
    d = bill_summaries.groupby(['bill.type', 'bill.number'])[c].nunique().value_counts()
    print(d)

actionDate
1    2725
2       9
4       1
Name: count, dtype: int64
actionDesc
1    2722
2      12
5       1
Name: count, dtype: int64
currentChamber
1    2728
2       6
3       1
Name: count, dtype: int64
currentChamberCode
1    2728
2       6
3       1
Name: count, dtype: int64
lastSummaryUpdateDate
1    2722
2      12
5       1
Name: count, dtype: int64
text
1    2730
2       4
4       1
Name: count, dtype: int64
updateDate
1    2722
2      12
5       1
Name: count, dtype: int64
versionCode
1    2722
2      12
5       1
Name: count, dtype: int64
bill.congress
1    2735
Name: count, dtype: int64
bill.number
1    2735
Name: count, dtype: int64
bill.originChamber
1    2735
Name: count, dtype: int64
bill.originChamberCode
1    2735
Name: count, dtype: int64
bill.title
1    2735
Name: count, dtype: int64
bill.type
1    2735
Name: count, dtype: int64
bill.updateDateIncludingText
1    2735
Name: count, dtype: int64
bill.url
1    2735
Name: count, dtype: int64


In [22]:
bills = bill_summaries[['bill.type', 'bill.number', 'bill.congress', 'bill.originChamber', 
                        'bill.originChamberCode', 'bill.title', 'bill.updateDateIncludingText', 'bill.url']].drop_duplicates()
bills.columns = [c.lower().replace('.', '_') for c in bills.columns]
bills.to_csv('data/thirdNF/bills.csv', index=False)

In [23]:
bill_versions = bill_summaries.drop(['bill.congress', 'bill.originChamber', 'bill.originChamberCode', 
                                     'bill.title', 'bill.updateDateIncludingText', 'bill.url'], axis=1)

In [24]:
bill_versions.columns = [c.lower().replace('.', '_') for c in bill_versions.columns]
bill_versions.to_csv('data/thirdNF/bill_versions.csv', index=False)

In [25]:
vote_compare.to_csv('data/thirdNF/vote_compare.csv', index=False)

In [26]:
bills2 = pd.merge(bills, sponsored_legislation,
                  left_on='bill_url',
                  right_on='url',
                  how='outer',
                  validate='one_to_one',
                  indicator='matched')

In [27]:
bills2['matched'].value_counts()

matched
right_only    11644
both           2735
left_only         0
Name: count, dtype: int64

In [28]:
pd.Series(['amendment' in x for x in bills2['url']]).value_counts()

False    10369
True      4010
Name: count, dtype: int64

In [29]:
bills2 = bills2[['bill_type', 'bill_number', 'bill_congress', 
                 'bill_originchamber', 'bill_originchambercode', 'bill_title', 
                 'bill_updatedateincludingtext', 'introducedDate', 
                 'url', 'bioguide_id']]

In [30]:
bills2.to_csv('data/thirdNF/bills.csv', index=False)

In [31]:
terms.head(3)

,bioguide_id,chamber,congress,stateCode,startYear,endYear,district
0,R000575,House of Representatives,108,AL,2003,2005,3.0
1,R000575,House of Representatives,109,AL,2005,2007,3.0
2,R000575,House of Representatives,110,AL,2007,2009,3.0


In [32]:
terms[terms[['bioguide_id', 'congress']].duplicated(keep=False)]

,bioguide_id,chamber,congress,stateCode,startYear,endYear,district
2449,S001150,House of Representatives,118,CA,2023,2024,28.0
2450,S001150,Senate,118,CA,2024,2025,NaN
2752,M000133,House of Representatives,113,MA,2013,2013,5.0
2753,M000133,Senate,113,MA,2013,2015,NaN
2801,W000437,House of Representatives,110,MS,2007,2007,1.0
2802,W000437,Senate,110,MS,2007,2009,NaN
2866,K000394,House of Representatives,118,NJ,2023,2024,3.0
2867,K000394,Senate,118,NJ,2024,2025,NaN
2918,G000555,House of Representatives,111,NY,2009,2009,20.0
2919,G000555,Senate,111,NY,2009,2011,NaN


In [33]:
terms.to_csv('data/thirdNF/terms.csv', index=False)

In [34]:
contrib.head(3).T

,0,1,2
contributor_name,FARMER'S & MERCHANTS BANK,MCLAUGHLIN AND ASSOCIATES,"DRUCKER LAWHON, LLP"
contributor_aggregate_ytd,920.34,40869.75,38500.0
memo_text,NaN,NaN,NaN
pdf_url,http://docquery.fec.gov/cgi-bin/fecimg/?139606...,http://docquery.fec.gov/cgi-bin/fecimg/?259710...,https://docquery.fec.gov/cgi-bin/fecimg/?20221...
fec_committee_id,NaN,NaN,NaN
fec_id,H2AL03032,H2AL03032,H2AL03032


In [35]:
contrib.to_csv('data/thirdNF/contrib.csv', index=False)

In [36]:
import pandas as pd

def pandas_df_to_dbml(df: pd.DataFrame, table_name: str) -> str:
    """
    Converts a pandas DataFrame to a DBML string.

    Args:
        df: The pandas DataFrame to convert.
        table_name: The name of the table in the DBML schema.

    Returns:
        A DBML string representing the DataFrame schema.
    """

    dbml_string = f"Table {table_name} {{\n"

    for column_name, column_type in df.dtypes.items():
        dbml_type = map_pandas_dtype_to_dbml_type(column_type)
        dbml_string += f"  {column_name} {dbml_type}\n"

    dbml_string += "}\n"
    return dbml_string

def map_pandas_dtype_to_dbml_type(dtype) -> str:
    """Maps a pandas dtype to a DBML type."""
    dtype_name = str(dtype)
    if "int" in dtype_name:
      return "int"
    if "float" in dtype_name:
      return "float"
    if "datetime" in dtype_name:
        return "datetime"
    return "varchar"

In [37]:
print(pandas_df_to_dbml(bill_versions, 'bill_versions'))

Table bill_versions {
  actiondate varchar
  actiondesc varchar
  currentchamber varchar
  currentchambercode varchar
  lastsummaryupdatedate varchar
  text varchar
  updatedate varchar
  versioncode int
  bill_number int
  bill_type varchar
}



In [38]:
print(pandas_df_to_dbml(bills2, 'bills'))

Table bills {
  bill_type varchar
  bill_number float
  bill_congress float
  bill_originchamber varchar
  bill_originchambercode varchar
  bill_title varchar
  bill_updatedateincludingtext varchar
  introducedDate varchar
  url varchar
  bioguide_id varchar
}



In [39]:
print(pandas_df_to_dbml(members, 'members'))

Table members {
  bioguide_id varchar
  full_name varchar
  birthyear float
  image varchar
  office_address varchar
  phone varchar
  website varchar
  fec_id varchar
  bioname varchar
  chamber varchar
  left_right_ideology float
  state_abbrev varchar
  district_code int
  icpsr int
  party varchar
}



In [40]:
print(pandas_df_to_dbml(terms, 'terms'))

Table terms {
  bioguide_id varchar
  chamber varchar
  congress int
  stateCode varchar
  startYear int
  endYear int
  district float
}



In [41]:
print(pandas_df_to_dbml(vote_compare, 'vote_compare'))

Table vote_compare {
  bioname varchar
  comparison_member varchar
  agree float
}

